# 📋Table of Contents
* [INtroduction](#1)
* [📚 Import Libraries](#2)
* [📖 Read Dataset](#3)
* [😅😅Missing & Duplicate Values](#4) 
    - [Missing Values](#4.4)
    - [Duplicates](#4.5)
* [🖇🖇📭Data Pipeline](#7)
    - [Creating X and y](#7.1)
    - [Okay I need to explain](#7.2)
    - [Main Pipeline](#7.3)
* [⚙️Finding and Training Models With Best Hyperparameters](#8)
    - [Random Jungle](#8.1)
    - [My Cat](#8.2)
    - [LGBM LIGHT](#8.3)
    - [Extreme Guy](#8.4)
* [👀👁FInal MOdel WIth VOting](#9)
* [📊Generate Predictions and Create Submission File](#10)

# <center><a id = 1>INtroduction
Hi , you clicked this notebook, so without any wasting time, let's get started..., Assuming (may be true or not) you're a beginner who needs a nice notebook to experiment with this  by forking it (so don't forget to upvote this)...
   
But some context.....
![https://miro.medium.com/v2/resize:fit:1400/1*WqId29D5dN_8DhiYQcHa2w.png](https://miro.medium.com/v2/resize:fit:1400/1*WqId29D5dN_8DhiYQcHa2w.png)    

# Why do customers churn?
Here are four common reasons customers can churn:

1. Price: by far, one of the most common reasons customers churn is price. Asking customers to pay top dollar for a product – especially if they aren’t experiencing the value to make it worth it – can be a stretch and quickly wear thin with decision-makers.

2. Product: many times, customers purchase a product based on what it can do (or may be able to do in the future) to make their lives easier. If this product isn’t living up to expectations, or if promised features aren’t being delivered, the customer will churn.

3. Competition: if you’re a SaaS organization, there will always be someone out there delivering the same thing to customers like you. If the competition becomes too appealing, there is often little you can do to pull a customer back in.

4. State of business: sometimes, factors at play internally make it impossible for a customer to continue their vendor relationships. However, this reason cuts just as deep as some of the others.

**The bank customer churn dataset is a commonly used dataset for predicting customer churn in the banking industry. It contains information on bank customers who either left the bank or continue to be a customer. The dataset includes the following attributes:**

1. Customer ID: A unique identifier for each customer
2. Surname: The customer's surname or last name
3. Credit Score: A numerical value representing the customer's credit score
4. Geography: The country where the customer resides (France, Spain or Germany)
5. Gender: The customer's gender (Male or Female)
6. Age: The customer's age.
7. Tenure: The number of years the customer has been with the bank
8. Balance: The customer's account balance
9. NumOfProducts: The number of bank products the customer uses (e.g., savings account, credit card)
10. HasCrCard: Whether the customer has a credit card (1 = yes, 0 = no)
11. IsActiveMember: Whether the customer is an active member (1 = yes, 0 = no)
12. EstimatedSalary: The estimated salary of the customer
13. Exited: Whether the customer has churned (1 = yes, 0 = no)

# <center> <a id = 2>📚Import Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import optuna
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# <center> <a id = 3>📖Read Dataset

In [ ]:
os.chdir("../")

In [ ]:
test = pd.read_csv("artifacts\data_ingestion\\test.csv")
train = pd.read_csv('artifacts\data_ingestion\\train.csv')

In [ ]:
train.dtypes

# <center> <a id = 4>😅😅Missing & Duplicate Values 

# <a id = 4.4> Missing Values

In [ ]:
print(train.isnull().sum())
test.isnull().sum()

* hmm.... Some problem will be solved at data pipeline

# <a id = 4.3> Duplicates

In [ ]:
# Remove duplicated rows
print(train.duplicated().sum())
train = train.drop_duplicates()
train.shape

# <center> <a id = 5>EDA

In [ ]:
train.describe().T

In [ ]:
# Plotting every feature on the graph 

from sklearn.preprocessing import LabelEncoder
# Create a deep copy of train and name it data
data = train.copy()

label_encoder = LabelEncoder()
data['Geography'] = label_encoder.fit_transform(data['Geography'])
data['Gender'] = label_encoder.fit_transform(data['Gender'])
data['Surname'] = label_encoder.fit_transform(data['Surname'])

data.hist(bins=50, figsize=(20,15), legend = True)
plt.show()

WE need to utilize the categorical columns also ... we are LabelEncoding this and many people have broken bank account , they literally have zero money

In [ ]:
data.nunique() #Uniqueness of the feature

# <center> <a id = 7>🖇🖇📭Data Pipeline

# <a id = 7.1> Creating X and y 

In [ ]:
X = train.drop('Exited', axis=1)

y = train['Exited']

In [ ]:
label_encoder = LabelEncoder()

X['Surname_encoded'] = label_encoder.fit_transform(train['Surname'])
test['Surname_encoded'] = label_encoder.transform(test['Surname'])


# getting list of the numerical and categorical columns
num = X.select_dtypes(include=['int64', 'float64']).columns
col = X.select_dtypes(include=['object']).columns

# <a id = 7.2> Okay I need to explain
If you don't understand why I dropped first 'Surname' and after some processing added again, lemme explain:

1. You can't use the Surname in the OnehotEncoder '2932' unique names , it will create '2932' features , and we will we facing the [Curse of dimensionality](https://towardsdatascience.com/curse-of-dimensionality-a-curse-to-machine-learning-c122ee33bfeb)..
2. To avoid this we are using LabelEncoder , well what is the difference between [ONehot vs Label Encoder](https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/)??
3. SO , we dropped 'Surname' in the dataset and then applied LabelEncoder to convert catt.. to numm. columns type , and added back 
4. Why we are doing this ??? Well, OnehotEncoder outperforms the LabelEncoder in creating useful features (so we will be using it in the main Pipeline), but when features are too much LabelEncoder is better choice , OnehoteEncoder is good to go when you have 10-20 unique values... but we have '2932'

# <a id = 7.3>Main Pipeline

In [ ]:
# Preprocessing for numerical data: imputation and scaling
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())])

# Preprocessing for categorical data: imputation and one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num),
        ('cat', categorical_transformer, col)])

#preprocessor is the final pipeline, we will using it before the data feds into the ml algo.... using Pipeline

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# <center> <a id = 8>⚙️Finding and Training Models With Best Hyperparameters

# What is OPtuna??
Hmmmm.. background we need models fine tuned (meaning with best Hyperparameters) , so we are using Optuna

[OPtuna](https://towardsdatascience.com/state-of-the-art-machine-learning-hyperparameter-optimization-with-optuna-a315d8564de1) is a software framework that automatically optimizes hyperparameters. It's designed for machine learning and can be used with other frameworks like TensorFlow, PyTorch, Keras, and SKlearn. 
Using OPtuna is going to help us to find best hyperparameters

# <a id = 8.1> Random Jungle

[Random forests](https://www.analyticsvidhya.com/blog/2021/06/understanding-random-forest/) or random decision forests is an ensemble learning method for classification, regression and other tasks that operates by constructing a multitude of decision trees at training time. For classification tasks, the output of the random forest is the class selected by most trees. For regression tasks, the mean or average prediction of the individual trees is returned ..

In [ ]:
# Don't uncomment to run it this will take more time than you can think and Kaggle allows , above 12 hours

# def objective(trial):
#     # Define the hyperparameters to be optimized
#     n_estimators = trial.suggest_int('n_estimators', 10, 1000, step=5)
#     max_depth = trial.suggest_int('max_depth', 5, 100)
#     min_samples_split = trial.suggest_int('min_samples_split', 2, 50)
#     min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 50)
#     max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
#     bootstrap = trial.suggest_categorical('bootstrap', [True, False])
#     criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    
#     # Create a Random Forest model with the specified hyperparameters
#     model = RandomForestClassifier(
#         n_estimators=n_estimators,
#         max_depth=max_depth,
#         min_samples_split=min_samples_split,
#         min_samples_leaf=min_samples_leaf,
#         max_features=max_features,
#         bootstrap=bootstrap,
#         criterion=criterion,
#         random_state=42
#     )
#     pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
#     # Train the model
#     pipeline.fit(X_train, y_train)
    
#     # Make predictions on the test set
#     predictions = pipeline.predict_proba(X_test)[:, 1]
    
#     # Calculate AUC-ROC as the metric to optimize
#     auc_roc = roc_auc_score(y_test, predictions)
    
#     return auc_roc

# # Create a study object and optimize the objective function
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=1500)

# # Get the best hyperparameters
# best_params = study.best_params
# print("Best Hyperparameters:", best_params)


In [ ]:
# # Train the final model with the best hyperparameters on the entire dataset
# best_model = RandomForestClassifier(**{'n_estimators': 425, 'max_depth': 13, 'min_samples_split': 41, 'min_samples_leaf': 2, 
#                                       'max_features': 'sqrt', 'bootstrap': False, 'criterion': 'entropy', 'random_state': 42})

# Ran_best = Pipeline(steps=[('preprocessor', preprocessor), ('model', best_model)])
    
# Ran_best.fit(X_train, y_train)

# predictions = Ran_best.predict_proba(X_test)[:, 1]

# auc_roc = roc_auc_score(y_test, predictions)
    
# auc_roc

#previous 0.8863970621948215
# after 0.851376404143134

# <a id= 8.2> My Cat

[CatBoost](https://www.analyticsvidhya.com/blog/2017/08/catboost-automated-categorical-data/) is an open-source machine learning library developed by Yandex. It's an algorithm for gradient boosting on decision trees that uses decision trees for classification and regression

In [ ]:
# Don't uncomment to run it this will take more time than you can think and Kaggle allows , above 12 hours

# from catboost import CatBoostClassifier

# def objective(trial):
#     iterations = trial.suggest_int('iterations', 50, 1000, step=10)
#     learning_rate = trial.suggest_float('learning_rate', 0.01, 0.5)
#     depth = trial.suggest_int('depth', 3, 15)
#     l2_leaf_reg = trial.suggest_float('l2_leaf_reg', 1e-3, 1)

#     model = CatBoostClassifier(
#         iterations=iterations,
#         learning_rate=learning_rate,
#         depth=depth,
#         l2_leaf_reg=l2_leaf_reg,
#         random_state=42,
#         verbose=0  # Set verbose to 0 to suppress CatBoost output
#     )
#     pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    
#     pipeline.fit(X_train, y_train)
#     probabilities = pipeline.predict_proba(X_test)[:, 1]  # Get the probability of the positive class
    
#     auc_roc = roc_auc_score(y_test, probabilities)
    
#     return auc_roc

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=1500)

# best_params = study.best_params
# print("Best Hyperparameters (CatBoost):", best_params)

In [ ]:
# Train the final model with the best hyperparameters on the entire dataset
best_model = CatBoostClassifier(**{'iterations': 830, 'learning_rate': 0.08238714339235984, 'depth': 5,
                                 'l2_leaf_reg': 0.8106903985997884, 'random_state': 42, 'verbose': 0})

Cat_best = Pipeline(steps=[('preprocessor', preprocessor), ('model', best_model)])

Cat_best.fit(X_train, y_train)

predictions = Cat_best.predict_proba(X_test)[:, 1]

auc_roc = roc_auc_score(y_test, predictions)
    
auc_roc
#previous 0.8893488306520324
# after  0.891890282220051

# <a id = 8.3 > LGBM LIGHT

[LightGBM](https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc), short for light gradient-boosting machine, is a free and open-source distributed gradient-boosting framework for machine learning, originally developed by Microsoft.

In [ ]:
# Don't uncomment to run it this will take more time than you can think and Kaggle allows , above 12 hours

# from lightgbm import LGBMClassifier

# def objective(trial):
#     n_estimators = trial.suggest_int('n_estimators', 50, 1000, step=10)
#     learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
#     max_depth = trial.suggest_int('max_depth', 3, 15)
#     min_child_samples = trial.suggest_int('min_child_samples', 1, 20)
#     subsample = trial.suggest_float('subsample', 0.5, 1.0)
#     colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)

#     model = LGBMClassifier(
#         n_estimators=n_estimators,
#         learning_rate=learning_rate,
#         max_depth=max_depth,
#         min_child_samples=min_child_samples,
#         subsample=subsample,
#         colsample_bytree=colsample_bytree,
#         random_state=42
#     )
#     pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    
#     pipeline.fit(X_train, y_train)
#     probabilities = pipeline.predict_proba(X_test)[:, 1]  # Get the probability of the positive class
    
#     auc_roc = roc_auc_score(y_test, probabilities)
    
#     return auc_roc

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=1500)

# best_params = study.best_params
# print("Best Hyperparameters (LightGBM):", best_params)

In [ ]:
# Train the final model with the best hyperparameters on the entire dataset
best_model = LGBMClassifier(**{'n_estimators': 960, 'learning_rate': 0.031725771326186744, 'max_depth': 8, 'min_child_samples': 8, 
                               'subsample': 0.7458307885861184, 'colsample_bytree': 0.5111460378911089, 'random_state': 42})

LGBM_best = Pipeline(steps=[('preprocessor', preprocessor), ('model', best_model)])
    
LGBM_best.fit(X_train, y_train)

predictions = LGBM_best.predict_proba(X_test)[:, 1]

auc_roc = roc_auc_score(y_test, predictions)
    
auc_roc
#previous 0.8899706366593071
#after 0.8932684835683227

# <a id = 8.4> Extreme Guy

[XGBoost](https://www.analyticsvidhya.com/blog/2018/09/an-end-to-end-guide-to-understand-the-math-behind-xgboost/), or eXtreme Gradient Boosting, is a machine learning algorithm that's used for supervised learning tasks like classification and regression. It's a popular algorithm because it can handle large datasets and perform well on many machine learning tasks, btw this is my boy..

In [ ]:
# Don't uncomment to run it this will take more time than you can think and Kaggle allows , above 12 hours

# def objective(trial):
#     n_estimators = trial.suggest_int('n_estimators', 50, 1000, step=10)
#     learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.5)
#     max_depth = trial.suggest_int('max_depth', 3, 15)
#     min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
#     gamma = trial.suggest_loguniform('gamma', 1e-3, 1)

#     model = XGBClassifier(
#         n_estimators=n_estimators,
#         learning_rate=learning_rate,
#         max_depth=max_depth,
#         min_child_weight=min_child_weight,
#         gamma=gamma,
#         random_state=42
#     )
#     pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    
#     pipeline.fit(X_train, y_train)
#     predictions = pipeline.predict_proba(X_test)[:, 1]
    
#     auc_roc = roc_auc_score(y_test, predictions)
    
#     return auc_roc

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=1500)

# best_params = study.best_params
# print("Best Hyperparameters (XGBoost):", best_params)

In [ ]:
# Train the final model with the best hyperparameters on the entire dataset
best_model = XGBClassifier(**{'n_estimators': 810, 'learning_rate': 0.07921079869615913, 'max_depth': 5,
                            'min_child_weight': 8, 'gamma': 0.27423983829634263, 'random_state': 42, 'objective': 'binary:logistic',
                            'eval_metric': 'auc', 'n_jobs': -1})

XGB_best = Pipeline(steps=[('preprocessor', preprocessor), ('model', best_model)])
    
XGB_best.fit(X_train, y_train)

predictions = XGB_best.predict_proba(X_test)[:, 1]

auc_roc = roc_auc_score(y_test, predictions)
    
auc_roc
#previous 0.8895683275081493
#after 0.8930475156029525

# <center> <a id = 9>👀👁FInal MOdel WIth VOting

A [Voting Classifier](https://medium.com/@imamitsingh/voting-classifiers-in-machine-learning-a532935fe592#:~:text=A%20Voting%20Classifier%20is%20a,chosen%20class%20as%20the%20output.) is a machine learning model that trains on an ensemble of numerous models and predicts an output (class) based on their highest probability of chosen class as the output..

In [ ]:

# # Create a VotingClassifier with the three XGBoost models
voting = VotingClassifier(estimators=[
    ('Model1', LGBM_best),
    ('Model2', XGB_best),
    ('Model3', Cat_best)
], voting='soft', weights = [0.5, 0.3, 0.2])

voting.fit(X_train, y_train)

predictions = voting.predict_proba(X_test)[:, 1]
predict = voting.predict(X_test)

auc_roc = roc_auc_score(y_test, predictions)
acuu = accuracy_score(y_test, predict) 
auc_roc, acuu

# with all 4 0.8898841550500115
#with best three 0.8901529119994704
# with best two 0.890129825391591
# with Cat & XGB 0.8898752572908788
# with LGBM & Cat 0.8900580892007095
# Stack two 0.8901571929273973
# Stack three 0.8901870809637238
# 0.8936971525454481, 0.8702259548090382

# <center> <a id = 10>📊Generate Predictions and Create Submission File

In [ ]:
voting.fit(X, y)

# Making predictions on the test set
pred = voting.predict_proba(test)[:, 1]

pred.to_csv('sample_submission.csv', index = False)

pred.head()